In [112]:
import numpy as np
import cmath

In [174]:
class platform:
    def __init__(self,d,D):
        "d: distancia de cada junta universal en la plataforma al centro"
        "D: distancia del eje de cada motor al centro"
        "la altura del eje del motor es conciderada el plano z=0"
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
    
        self.D=D

    def x_rotation_matrix(self,r_x):
        return np.array([[1,0,0],[0,np.cos(r_x),-np.sin(r_x)],[0,np.sin(r_x),np.cos(r_x)]]) 
    
    def y_rotation_matrix(self,r_y):
        return np.array([[np.cos(r_y),0,np.sin(r_y)],[0,1,0],[-np.sin(r_y),0,np.cos(r_y)]])
          
    def rotate(self,movement,joint,angle):
        if movement=="pitch":
            M_r=self.x_rotation_matrix(angle)
        if movement=="roll":
            M_r=self.y_rotation_matrix(angle)
            
        if joint=="C1":
            wm=np.identity(3)
        if joint=="C2":
            wm=self.get_m2_basis()
        if joint=="C3":
            wm=self.get_m3_basis()
        #transformamos los puntos a la base conveniente
        self.c1=np.linalg.solve(wm,self.c1)
        self.c2=np.linalg.solve(wm,self.c2)
        self.c3=np.linalg.solve(wm,self.c3)
        #realizamos la rotación de los mismos
        self.c1=M_r.dot(self.c1)
        self.c2=M_r.dot(self.c2)
        self.c3=M_r.dot(self.c3)
        #corregimos el offset de la plataforma
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #obtengo los puntos en la base original
        self.c1=wm.dot(self.c1)
        self.c2=wm.dot(self.c2)  
        self.c3=wm.dot(self.c3)
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
    def get_m2_basis(self):
        Wm2=np.array([[-np.cos(np.pi/3),np.cos(np.pi/6),0],[-np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm2

    def get_m3_basis(self):
        Wm3=np.array([[-np.cos(np.pi/3),-np.cos(np.pi/6),0],[np.sin(np.pi/3),-np.sin(np.pi/6),0],[0,0,1]])
        return Wm3
    
        
    def calculate_P(self):
            wm2=self.get_m2_basis()
            wm3=self.get_m3_basis()
            P1=self.c1+np.array([0,self.D,0])
            P2=np.linalg.solve(wm2,self.c2)+np.array([0,self.D,0])
            P3=np.linalg.solve(wm3,self.c3)+np.array([0,self.D,0])
            return P1,P2,P3
        
    def correct_height(self,h):
        "[TODO]: Hay que desarrollarlo"
        aux=np.array([0,0,h])
        self.c1=self.c1+aux
        self.c2=self.c2+aux
        self.c3=self.c3+aux
    def solve_platform(self,r_x,r_y,h):
        if r_x==0 and r_y==0:
            self.correct_height(h)
            return
        
        ang_vect=np.array([r_x,r_y,0])
        movement,joint=self.assign_movement(r_x,r_y)
        #obtengo la base más conveniente
        if joint=="C1":
            wm=np.identity(3)
        if joint=="C2":
            wm=self.get_m2_basis()
        if joint=="C3":
            wm=self.get_m3_basis()
        #referencio el vector de rotación a la base
        ang_vect=np.linalg.solve(wm,ang_vect)
        
        if movement=="roll":
            self.rotate(movement,joint,ang_vect[0])
        if movement=="pitch":
            self.rotate(movement,joint,ang_vect[1])
        self.correct_height(h)
        "[TODO]: hay que desarrollar la función para calcular el angulo de los motores"
        return self.calculate_P()
    
    def assign_movement(self,r_x,r_y):
        angle=cmath.polar(r_x+1j*r_y)[1]*(180/np.pi)
    
        if angle<0:
            angle+=180
        if angle > 165:
            return "roll","C1"
        if angle > 135 and angle <= 165:
            return "pitch","C2"
        if angle > 105 and angle <= 135:
            return "roll","C3"
        if angle > 75 and angle <= 105:
            return "pitch","C1"
        if angle <= 75 and angle >45:
            return "roll","C2"
        if angle <= 45 and angle >15:
            return "pitch","C3"  
        if angle <= 15:
            return "roll","C1"  


    def generate_geogebra (self):
        s1=geogebra_script()
        #Imprimo el plato
        s1.add_point(self.c1,"C1")
        s1.add_point(self.c2,"C2")
        s1.add_point(self.c3,"C3")
        s1.define_polygon("C1","C2","C3")
        #Imprimo los planos de acción de los motores
        s1.add_point(np.array([0,-1,0]),"AUX1")
        s1.add_point(np.array([-np.sin(np.pi/3),np.cos(np.pi/3),0]),"AUX2")
        s1.add_point(np.array([np.sin(np.pi/3),np.cos(np.pi/3),0]),"AUX3") 
        s1.add_point(np.array([0,0,0]),"AUX0") 
        s1.add_point(np.array([0,0,1]),"AUX4")
        s1.define_plane("AUX1","AUX0","AUX4")
        s1.define_plane("AUX2","AUX0","AUX4")
        s1.define_plane("AUX3","AUX0","AUX4")
        s1.print_script()

In [175]:
class geogebra_script:
    def __init__(self):
        self.script=""
        self.first_item=1
        
    def add_point(self,point,name):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\"{0}=({1:.3f},{2:.3f},{3:.3f})\"\n".format(name,point[0],point[1],point[2])
        
    def define_polygon(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Polygon({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"

    def define_plane(self,point1,point2,point3):
        if self.first_item==0:
            self.script+=","
        self.first_item=0
        self.script+="\""
        self.script+="Plane({},{}, {})".format(point1,point2,point3)
        self.script+="\""
        self.script+="\n"
        
    def print_script(self):
        print("Execute[{\n"+self.script+"}]")

In [176]:
np.identity(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [187]:
p1=platform(d=1,D=18)
p1.solve_platform(30*np.pi/180,30*np.pi/180,1)
p1.generate_geogebra()

Execute[{
"C1=(0.000,-1.000,0.672)"
,"C2=(-0.866,0.500,0.672)"
,"C3=(0.548,0.316,1.656)"
,"Polygon(C1,C2, C3)"
,"AUX1=(0.000,-1.000,0.000)"
,"AUX2=(-0.866,0.500,0.000)"
,"AUX3=(0.866,0.500,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(AUX1,AUX0, AUX4)"
,"Plane(AUX2,AUX0, AUX4)"
,"Plane(AUX3,AUX0, AUX4)"
}]


In [178]:
p1=platform(d=1,D=18)
p1.rotate(movement="pitch",joint="C3",angle=60*(np.pi/180))
p1.generate_geogebra()

Execute[{
"C1=(0.000,-1.000,0.433)"
,"C2=(-0.866,0.500,0.433)"
,"C3=(0.217,0.125,-0.866)"
,"Polygon(C1,C2, C3)"
,"AUX1=(0.000,-1.000,0.000)"
,"AUX2=(-0.866,0.500,0.000)"
,"AUX3=(0.866,0.500,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(AUX1,AUX0, AUX4)"
,"Plane(AUX2,AUX0, AUX4)"
,"Plane(AUX3,AUX0, AUX4)"
}]


In [179]:
np.pi/9

0.3490658503988659

In [141]:
assign_movement(-1,0)

('roll', 'C1')